In [ ]:
#Second test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [1]:
# Global Setup
# Creating temp local repos to test on

TMP=/tmp/temp-git-test


git clone https://github.com/LorenzoFerraces/bash-test.git "$TMP"

# Move to second local to push some changes
command cd "$TMP" 
git pull origin main 
echo "hello second local" > hello.txt
git add hello.txt
git commit -m "first commit on second local"
git push


Cloning into '/tmp/temp-git-test'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 219 (delta 34), reused 3 (delta 0), pack-reused 182
Receiving objects: 100% (219/219), 18.38 KiB | 495.00 KiB/s, done.
Resolving deltas: 100% (114/114), done.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
[main 0714c05] first commit on second local
 1 file changed, 1 insertion(+)
 create mode 100644 hello.txt
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 6 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 310 bytes | 310.00 KiB/s, done.
Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   aabb100..0714c05  main -> main


In [2]:
# git-rename-file tests
# Assert file doesn't exist
[ ! -f "newHello.txt" ]; echo $?;

0


In [3]:
export GIT_MESSAGE="renamed"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
contents="$(cat hello.txt)"
git-rename-file hello.txt newHello.txt

issuing: git mv hello.txt newHello.txt
issuing: git add newHello.txt

GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git commit -m 'renamed'
    [main f71d1ce] renamed
     1 file changed, 0 insertions(+), 0 deletions(-)
     rename hello.txt => newHello.txt (100%)
GIT_NO_CONFIRM set to 1, so skipping confirmation
issuing: git push --verbose
Pushing to https://github.com/LorenzoFerraces/bash-test.git
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 6 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (2/2), 271 bytes | 271.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
POST git-receive-pack (444 bytes)
To https://github.com/LorenzoFerraces/bash-test.git
   0714c05..f71d1ce  main -> main
updating local tracking ref 'refs/remotes/origin/main'
[main f71d1ce] renamed
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename hello.txt => newHello.txt (100%)

issuing: git reset HEAD hello.txt


In [4]:
# Asserts
[ -f "newHello.txt" ] && [ "$(cat newHello.txt)" == "$(echo $contents)" ]; 
echo $?;

0


In [5]:
# git-restore-staged-alias test
echo "second change on local" >> newHello.txt
git add newHello.txt

In [6]:
# Assert theres is one staged file 
[ $(git status -s | grep -c "^[MTADRCU]") -eq 1 ]; echo $?; 

0


In [ ]:
git-restore-staged-alias newHello.txt

issuing: cp -vpf newHello.txt _git-trash


In [8]:
# Assert theres are none staged files
[ $(git status -s | grep -c "^[MTADRCU]") -eq 0 ]; echo $?; 
[ $(git status -s | grep -c "^\?") -eq 0 ]; echo $?; 

0
0


In [ ]:
# git-revert-file-alias tests
echo "second change on local" >> newHello.txt


In [ ]:
# Assert theres is 1 unstaged file
[ $(git status -s | grep -c "^[MTADRCU]") -eq 0 ]; echo $?; 

In [ ]:
# undo changes and remove newly created files

# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null
git rm -f *.txt > /dev/null
git add . > /dev/null
git commit -m "cleanup test files"
git push

In [ ]:
# delete test local repos

command cd "/tmp"
rm -rf temp-git-test > /dev/null
export GIT_NO_CONFIRM=$previous
